# Chapter 5: Pandas

While pandas adopts many coding idioms from NumPy, the biggest difference is that pandas is designed for working with tabular or heterogeneous data. NumPy, by contrast, is best suited for working with homogeneous numerical array data. Since becoming an open source project in 2010, pandas has matured into a quite large library that’s applicable in a broad set of real-world use cases. The developer community has grown to over 800 distinct contributors, who’ve been helping build the project as they’ve used it to solve their day-to-day data problems. Throughout the rest of the book, I use the following import convention for pandas:

In [1]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web

## Introduction to pandas datastructure

To get started with pandas, you will need to get comfortable with its two workhorse data structures: Series and DataFrame. While they are not a universal solution for every problem, they provide a solid, easy-to-use basis for most applications.

### Series

A series is a one-dimensional array-like object containing a sequence of values (of similar types to numpy types) and as an associated array of data labels, called its *index*. The simplest Series is formed from only an array of data.

In [2]:
obj = pd.Series([4, 7, -5, 3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

The string representation of a Series displayed interactively shows the index on the left and the values on the right. Since we did not specify an index for the data, a default one consisting of the integers $0$ through $N - 1$ is created. You can get the array representation and index object of the Series via its values and index attributes respecitvely:

In [3]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [4]:
obj.index

RangeIndex(start=0, stop=4, step=1)

Often it is desirable to create a Series with an index identifying each data point with a label:

In [5]:
obj2 = pd.Series([4, 7, -5, 3], index=['d', 'b', 'a', 'c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

Compared with numpy arrays, you can use labels in the index when selecting single values or a set of values:

In [6]:
obj2['a']

-5

In [7]:
obj2['d'] = 6
obj2[['c', 'a', 'd']]

c    3
a   -5
d    6
dtype: int64

Here the indexes is interpreted as a list of indices, even though it contains strings instead of integers.

Using NumPy unctions or NumPy-like opreations, such as filtering with a boolean array, scalar multiplication, or applying math functions, will preserve the index-value link:

In [8]:
obj2[obj2 > 0]

d    6
b    7
c    3
dtype: int64

In [9]:
obj2  * 2

d    12
b    14
a   -10
c     6
dtype: int64

In [10]:
np.exp(obj2)

d     403.428793
b    1096.633158
a       0.006738
c      20.085537
dtype: float64

Another way to think about a Series is as a fixed-length, ordered dictionary, as it is mapping of index values to data values. It can be used in many contexts where you might use a dict.

In [11]:
'b' in obj2

True

In [12]:
'e' in obj2

False

In [13]:
sdata = {'Ohio' : 35000, 'Texas' : 71000, 'Oregon' : 16000, 'Utah' : 50000}
obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah      50000
dtype: int64

When you are only passing a dictionary, the index in the resulting Series will have the dictionarys keys in sortedorder. You can override this by passing the dict keys in the order you want them to apear in the resulting Series:

In [14]:
states = ['California', 'Ohio', 'Oregon', 'Texas']
obj4 = pd.Series(sdata, index=states)
obj4

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

Here, three values found in *sdata* were placed in the appropriate locations, but since no value for 'California' was found, it appears as NaN (not a number), which is considered in pandas to mark missing og NA values. Since 'Utah' was not included in states, it is excluded from the resulting object.

In [15]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [16]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

Series also has these as instance methods:

In [17]:
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

A useful Series feature for many applications is that it automatically aligns by index label in arithmetic operations:

In [18]:
obj4 + obj3

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

This can be thought of as a join operation in database terms.

A series's index can be altered in-place by assignment:

In [19]:
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [20]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### DataFrame

A DataFrame represents a rectangular table of data and contains an ordered collection of columns, each of which can be a different value type (numeric, string, boolean, etc). The DataFrame has both a row and columnd index; it can be thought of as a dictionary of Series all sharing the same index. Under the hood, the data is stored as one or more two-dimensional blocks rather than a list, cit or some other collection of one-dimensional arrays. The exact details of DataFrame's are outside the scope of this book. There are many ways to construct a Data Frame, though one of the most common is from a dictionary of equal-length lists or Numpy Arrays.

In [21]:
data = {'state' : ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada', 'Nevada'],
        'year' : [2000, 2001, 2002, 2001, 2002, 2003],
        'pop' : [1.5, 1.7, 3.6, 2.4, 2.9, 2.3]}
frame = pd.DataFrame(data)

The resulting DataFrame will have its index assigned automatically as with Series, and the columns are placed in sorted order:

In [22]:
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,2.3


If you are using the Jupyter notebook, pandas DataFrame object will be displayer as a more browser-friendly HTML table. For large DataFrame, the *head* method selects only the first five rows:

In [23]:
frame.head()

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


If you specify a sequence of columns, the DataFrame's columns will be arranged in that order:

In [24]:
pd.DataFrame(data, columns=['year', 'state', 'pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,2.3


If you pass a column that isn't contained in the dict, it will appear with missing values in the results:

In [25]:
frame2 = pd.DataFrame(data, columns=['year', 'state', 'pop', 'debt'],
                            index = ['one', 'two', 'three', 'four', 'five', 'six'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,2.3,NaN


In [26]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

A column in a DataFrame can be retrieved as a Series either by dict-like notation or by attribute:

In [27]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [28]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

Rows can also be retrieved by position or name with the special *loc* attribute

In [29]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

Columns can be modified by assignment. For example, the empty 'debt' column could be assigned a scalar value or an array of values:

In [30]:
frame2['debt'] = 16.5
frame2 

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,2.3,16.5


In [31]:
frame2['debt'] = np.arange(6.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,2.3,5.0


When you are assigning lists or arrays to a column, the value's length must match the length of the DataFrame. If you assign a Series, its labels will be realign exactly to the DataFrame's index, inserting missing valuesin any holes:

In [32]:
val = pd.Series([-1.2, -1.5, -1.7], index = ['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,2.3,NaN


Assigning a column that does not exist will create a new column. The del keyword will delete columns as with a dict.

As an example of del:

In [33]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,2.3,NaN,False


The del method can then be used to remove this column:

In [34]:
del frame2['eastern']
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

Another common form of data is a nested dict of dicts:

In [35]:
pop = {'Nevada' : {2001 : 2.4, 2002 : 2.9},
       'Ohio' : {2000: 1.5, 2001 : 1.7, 2002 : 3.6}}

If the nested dict is passed to the DataFrame, pandas will interpret the outer dict keys as the columns and the inner keys as the row indices:

In [36]:
frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


You can transpose the DataFrame (swap rows and columns) with similar syntax to a NumPy array:

In [37]:
frame3.T

,2001,2002,2000
Nevada,2.4,2.9,NaN
Ohio,1.7,3.6,1.5


The keys in the inner dictionaries are combined and sorted to form the index in the result. This isnt true if an explicit index is specified:

In [38]:
pd.DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


Dict of Series are treated in much the same way:

In [39]:
pdata = {'Ohio' : frame3['Ohio'][:-1],
         'Nevada' : frame3['Nevada'][:2]}
pd.DataFrame(pdata)

,Ohio,Nevada
2001,1.7,2.4
2002,3.6,2.9


If a DataFrame's index and columns have their name attributes set, these will also be displayed:

In [40]:
frame3.index.name = 'year'; frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


As with Series, the values attribute returns the data contained in the DataFrame as a two-dimensional ndarray:

In [41]:
frame3.values

array([[2.4, 1.7],
       [2.9, 3.6],
       [nan, 1.5]])

if the DataFrame's columns are different dtypes, the dtype of the values array will be chosen to accomodate all of the columns: 

In [42]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7],
       [2003, 'Nevada', 2.3, nan]], dtype=object)

### Index Objects

Index objects are responsible for holding the axis labels and other metadata (like the axis name or names). Any array or other sequence of labels you use when construction a Series or DataFrame is internally converted to an Index:

In [43]:
obj = pd.Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

In [44]:
index[1:]

Index(['b', 'c'], dtype='object')

Index object are immutable and thus cant be modified by the user:

In [45]:
index[1] = 'd'

TypeError: Index does not support mutable operations

Immutability makes it safer to share Index object among data structures:

In [46]:
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [47]:
obj2 = pd.Series([1.5, -2.5, 0], index=labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [48]:
obj2.index is labels

True

In addition to being array-like, an Index also behaves like a fixed-size set:

In [49]:
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


In [50]:
frame3.columns

Index(['Nevada', 'Ohio'], dtype='object', name='state')

In [51]:
'Ohio' in frame3.columns

True

In [52]:
2003 in frame3.index

False

Unlike python sets, a pandas Index can contain duplicate labels:

In [53]:
dup_labels = pd.Index(['foo', 'foo', 'bar', 'bar'])
dup_labels

Index(['foo', 'foo', 'bar', 'bar'], dtype='object')

## Essential Functionality

### Reindexing

An important method on pandas objects is *reindex*, which means to create a new object with the data *conformed* tp a new index. Consider an example:

In [54]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

Calling *reindex* on this Series rearranges the data according to the new index, introdicing missing values if any index values were not already present:

In [55]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

For ordered data like time series, it may be desirable to do some interpolation or filling of values when reindexing. The method option allows us to do this, using a method such as *ffil*, which forward-fills the values:

In [56]:
obj3 = pd.Series(['blue', 'purple', 'yellow'], index = [0, 2, 4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [57]:
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

With DataFrame, reindex can can alter either the (row) index, columns or both. When passed only a sequence, it reindexes the rows in the result:

In [58]:
frame =pd.DataFrame(np.arange(9).reshape((3,3)), 
                    index = ['a', 'c', 'd'],
                    columns = ['Ohio', 'Texas', 'California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [59]:
frame2 = frame.reindex(['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


In [60]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns = states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


### Dropping entries from an axis

Dropping one or more entries from an axis is easy if you already have an index array or list without those entries. As that can require a bit of munging and set logic, the drop method will return a new object with the indicated value or values deleted from an axis:

In [61]:
obj = pd.Series(np.arange(5.), index = ['a', 'b', 'c', 'd', 'e'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
e    4.0
dtype: float64

In [62]:
new_obj = obj.drop('c')
new_obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

With DataFrame, index values can be deleted from either axis. To illustrate this, we first create an example DataFrame:

In [63]:
data = pd.DataFrame(np.arange(16).reshape((4,4)),
                    index = ['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns = ['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Calling *drop* with a sequence of labels will drop values from the row labels (axis = 0):

In [64]:
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


You can drop values from the columns by passing axis = 1 or axis = 'columns'

In [65]:
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


Many functions, like drop, which modify the size or shape of a Series or DataFrame, can manipulate an object *in-place* without returning a object:

In [66]:
obj.drop('c', inplace = True)
obj

a    0.0
b    1.0
d    3.0
e    4.0
dtype: float64

### Indexing, Selection and Filtering

Seres indexing works analogously to NumPy array indexing, except you can use the Series's index values instead of only integers. Here are som examples of this:

In [67]:
obj = pd.Series(np.arange(4.), index = ['a', 'b', 'c', 'd'])
obj

a    0.0
b    1.0
c    2.0
d    3.0
dtype: float64

In [68]:
obj['b']

1.0

In [69]:
obj[1]

1.0

In [70]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

Slicing with labels behaves differently than normal Python slicing in that the endpoint is inclusive:

In [71]:
obj['b':'c']

b    1.0
c    2.0
dtype: float64

Boolean selection:

In [72]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


### Selecting with loc and iloc

For DataFrame label-indexing on the rows, I introduce the special indexing operators *loc* and *iloc*. They enable you to select a subset of the rows and columns from a DataFrame with NumPy-like notation using either acis labels (loc) or integers (iloc). 

As a preliinary example, let's select a single row and multiple columns by label:

In [73]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int32

We'll then perform some similar sections with integers using *iloc*:

In [74]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int32

In [75]:
data.iloc[2]

one       8
two       9
three    10
four     11
Name: Utah, dtype: int32

### Integer indexes

Indexing is slightly different than built-in Python data structures.

In [76]:
ser = pd.Series(np.arange(3.))
ser
ser[-1]

KeyError: -1

with non-integer index, this is not a problem:

In [77]:
ser2 = pd.Series(np.arange(3.), index = ['a', 'b', 'c'])
ser2[-1]

2.0

### Arithmetic and Data Alignment

An important pandas feature for some applications is the behavior of arithmetic between objects, if any index pairs are note the same, the respective index in the result will be the union of the index pairs. For users with database experience, this is similar to an automatic outer join on the index labels. Let's look at an example:

In [78]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index = ['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4, 3.1], index = ['a', 'c', 'e', 'f', 'g'])
s1

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64

In [79]:
s2

a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64

In [80]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

The internal data alignment introduces missing values in the label locations that dont overlap. Missing values will then propagate in further arithmetic computations.

In the case of DataFrame, alignment is performed on both the rows and the columns:

In [81]:
df1 = pd.DataFrame(np.arange(9.).reshape((3, 3)), 
                   columns = list('bcd'),
                   index = ['Ohio', 'Texas', 'Colorado'])

df2 = pd.DataFrame(np.arange(12.).reshape((4, 3)),
                   columns = list('bde'),
                   index = ['Utah', 'Ohio', 'Texas', 'Oregon'])
                   
df1

,b,c,d
Ohio,0.0,1.0,2.0
Texas,3.0,4.0,5.0
Colorado,6.0,7.0,8.0


In [82]:
df2

,b,d,e
Utah,0.0,1.0,2.0
Ohio,3.0,4.0,5.0
Texas,6.0,7.0,8.0
Oregon,9.0,10.0,11.0


Adding these together returns a DataFrame whose index and columns are the unions of the ones in each DataFrame:

In [83]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


### Function Application and Mapping

Numpy ufunc (element-wise array methods) also work with pandas objects:

In [84]:
frame = pd.DataFrame(np.random.randn(4, 3), 
                     columns=list('bde'),
                     index = ['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.370452,-0.263048,-0.429274
Ohio,-0.156069,-0.610773,-0.206284
Texas,-0.536199,-0.599248,1.036966
Oregon,-0.380274,-0.031395,-0.228662


In [85]:
np.abs(frame)

,b,d,e
Utah,0.370452,0.263048,0.429274
Ohio,0.156069,0.610773,0.206284
Texas,0.536199,0.599248,1.036966
Oregon,0.380274,0.031395,0.228662


Another frequent operation is applying a function on one-dimensional arrays to each column or row. DataFrame's apply method does exactly this.

In [86]:
f = lambda x : x.max() - x.min()
frame.apply(f)

b    0.906651
d    0.579378
e    1.466240
dtype: float64

In [87]:
frame.apply(f, axis = 'columns')

Utah      0.799726
Ohio      0.454704
Texas     1.636214
Oregon    0.348879
dtype: float64

Many of the most common array statistics are DataFrame methods, so using apply is not necessary. The function passed to apply need not return a scalar value, it can also return a Series with multiple values:

In [88]:
def f(x):
    return pd.Series([x.min(), x.max()], index = ['min', 'max'])

frame.apply(f)

,b,d,e
min,-0.536199,-0.610773,-0.429274
max,0.370452,-0.031395,1.036966


Element-wise Python functions can be used, too. Suppose you wanted to compute a formatted string from each floating-point value in frame. You can do this with applymap:

In [89]:
format = lambda x : '%.2f' % x

frame.applymap(format)

,b,d,e
Utah,0.37,-0.26,-0.43
Ohio,-0.16,-0.61,-0.21
Texas,-0.54,-0.60,1.04
Oregon,-0.38,-0.03,-0.23


The reason for the name applymap is that Series has a map method for applying an element-wise function:

In [90]:
frame['e'].map(format)

Utah      -0.43
Ohio      -0.21
Texas      1.04
Oregon    -0.23
Name: e, dtype: object

### Sorting and Ranking

Sorting a dataset by some criterion is another important built-in operation. To sort lexicographically by row or column index, use the *sort_index* method, which returns a new, sorted object:

In [91]:
obj = pd.Series(range(4), index = list('dabc'))
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

With a DataFrame, you can sort by index on either axis:

In [92]:
frame = pd.DataFrame(np.arange(8).reshape((2, 4)),
                     index = ['three', 'one'],
                     columns = list('dabc'))
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [93]:
frame.sort_index(axis = 1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


## Summarizing and Computing Descriptive Statistics

pandas objects are equipped with a set of common mathematical and statistical methods. Most of these fall into the category of reductions or summary statistics, methods that extract a single value from a Series or a Series of values from the rows or columns of a DataFrame. Compared with the similar methods found on NumPy arrays, they have built-in handling for missing data. Consider a small DataFrame:

In [94]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5], 
                   [np.nan, np.nan], [0.75, -1.3]],
                   index = list('abcd'),
                   columns = ['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


Calling a DataFrame's sum method returns a Series containing column sums:

In [95]:
df.sum()

one    9.25
two   -5.80
dtype: float64

In [96]:
df.sum(axis = 'columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

Other methods are *accumulations*

In [97]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


In [98]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


### Correlation and Covariance

Some summary statistics, like correlation and covariance, are computed from pairs of arguments. Let's consider some DataFrames of stock prices and volumes obtained from Yahoo! Finance using the add-on pandas-datareader package.

In [100]:
all_data = {ticker : web.get_data_yahoo(ticker) for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']}
price = pd.DataFrame({ticker : data['Adj Close'] for ticker, data in all_data.items()})
volume = pd.DataFrame({ticker : data['Volume'] for ticker, data in all_data.items()})

Compute percent changes of the prices.

In [101]:
returns = price.pct_change()
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2020-09-03,-0.080061,-0.029100,-0.061947,-0.050015
2020-09-04,0.000662,-0.017276,-0.014036,-0.030941
2020-09-08,-0.067295,-0.008913,-0.054096,-0.036863
2020-09-09,0.039887,0.008663,0.042584,0.016034
2020-09-10,0.016025,0.009161,0.011856,0.007274


The corr method of Series computes the correlation of the overlapping, non-NA, aligned-by-index values in two Series. Relatedly, cov computes the covariance.

In [102]:
returns['MSFT'].corr(returns['IBM'])

0.5786348013637765

In [103]:
returns['MSFT'].cov(returns['IBM'])

0.00016201481746841009

### Unique Values, Value Counts and Membership

Another class of related methods extracts information about the values contained in a one-dimensional Series. To illustrate these, consider this example:

In [109]:
obj = pd.Series(list('cadaabbcc'))
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)